In [3]:
import neptune.new as neptune
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

project = neptune.get_project('ygorcanalli/LabelNoise')
df = project.fetch_runs_table(tag='hyperparameter-rerun').to_pandas()

In [4]:
df["model/params/noise-args"].replace('[0.0]', '0.0', inplace=True)
df["model/params/noise-args"].replace('[0]', '0.0', inplace=True)
df["model/params/noise-args"].replace('[0.05]', '0.05', inplace=True)
df["model/params/noise-args"].replace('[0.1]', '0.1', inplace=True)
df["model/params/noise-args"].replace('[0.15]', '0.15', inplace=True)
df["model/params/noise-args"].replace('[0.2]', '0.2', inplace=True)
df["model/params/noise-args"].replace('[0.25]', '0.25', inplace=True)
df["model/params/noise-args"].replace('[0.3]', '0.3', inplace=True)
df["model/params/noise-args"] = df["model/params/noise-args"].astype('float')

df.to_csv('raw_data.csv')

In [5]:
df.head()

,sys/creation_time,sys/description,sys/hostname,sys/id,sys/modification_time,sys/name,sys/owner,sys/ping_time,sys/running_time,sys/size,...,parameters/hyperparameters/num_layers,parameters/num-classes,parameters/num-features,source_code/entrypoint,source_code/git,monitoring/cpu,monitoring/gpu,monitoring/gpu_memory,monitoring/memory,monitoring/stdout
0,2021-04-04 22:31:20.055000+00:00,,meriadoc,LAB-423,2021-04-04 22:35:41.063000+00:00,synthetic-100-5-500K-uniform-[0.05],ygorcanalli,2021-04-04 22:35:37.510000+00:00,257.439,4808390.0,...,3.0,5.0,100.0,main.py,0f0b3e1dca1ee35dbc055f5a45002c97adf5a273,26.4,8.0,5.687683,11.029408,https://ui.neptune.ai/ygorcanalli/LabelNoise/e...
1,2021-04-02 22:37:10.263000+00:00,,meriadoc,LAB-368,2021-04-02 22:37:23.442000+00:00,synthetic-50-2-100K-uniform-[0.3],ygorcanalli,2021-04-02 22:37:21.276000+00:00,11.012,244885.0,...,2.0,2.0,50.0,main.py,877f5f997285690780c2e680ad9997a98baaf159,23.0,NaN,NaN,10.144955,https://ui.neptune.ai/ygorcanalli/LabelNoise/e...
2,2021-04-04 17:27:49.070000+00:00,,meriadoc,LAB-411,2021-04-04 17:29:26.729000+00:00,synthetic-80-5-200K-uniform-[0.3],ygorcanalli,2021-04-04 17:29:22.311000+00:00,93.237,1926275.0,...,3.0,5.0,80.0,main.py,0f0b3e1dca1ee35dbc055f5a45002c97adf5a273,19.5,NaN,NaN,6.535912,https://ui.neptune.ai/ygorcanalli/LabelNoise/e...
3,2021-04-04 17:10:07.670000+00:00,,meriadoc,LAB-405,2021-04-04 17:11:18.658000+00:00,synthetic-80-5-200K-uniform-[0.2],ygorcanalli,2021-04-04 17:10:49.525000+00:00,41.854,1926035.0,...,3.0,5.0,80.0,main.py,0f0b3e1dca1ee35dbc055f5a45002c97adf5a273,35.8,NaN,NaN,9.647404,https://ui.neptune.ai/ygorcanalli/LabelNoise/e...
4,2021-04-04 17:02:57.400000+00:00,,meriadoc,LAB-400,2021-04-04 17:04:04.442000+00:00,synthetic-80-5-200K-uniform-[0.15],ygorcanalli,2021-04-04 17:03:39.164000+00:00,41.763,1926037.0,...,3.0,5.0,80.0,main.py,0f0b3e1dca1ee35dbc055f5a45002c97adf5a273,36.0,NaN,NaN,9.223026,https://ui.neptune.ai/ygorcanalli/LabelNoise/e...


In [6]:
datasets = df["model/params/dataset"].unique()
robust_methods = ['none', 'boot-hard', 'boot-soft', 'forward']#df["model/params/robust-method"].unique()
noise_rates = df["model/params/noise-args"].unique()
noise_rates.sort()

In [7]:
def get_metric_by_dataset(dataset, metric):

    result = np.zeros( (len(robust_methods), len(noise_rates)) )
    for i, robust_method in enumerate(robust_methods):
        for j, noise_rate in enumerate(noise_rates):
            robust_method_index = df["model/params/robust-method"] == robust_method
            noise_rate_index = df["model/params/noise-args"] == noise_rate
            dataset_index = df["model/params/dataset"] == dataset
            df_index = robust_method_index & noise_rate_index & dataset_index
            result[i,j] = df[df_index][metric]

    return result

In [8]:
def plot_by_dataset(result, dataset, metric):
    plt.style.use('seaborn-whitegrid')
    fig = plt.figure()
    ax = plt.axes()
    for i, robust_method in enumerate(robust_methods):
        ax.plot(noise_rates, result[i,:], label=robust_method)
    ax.set_title("%s on %s dataset" % (metric, dataset))
    ax.set_xlabel("noise rates")
    ax.set_ylabel(metric);
    ax.legend()

    fig.show()
    fig.savefig("%s %s.png" % (dataset, metric), dpi=300)
    plt.close(fig)

In [9]:
for d in datasets:
    result =  get_metric_by_dataset(d, 'metrics/eval_accuracy')
    plot_by_dataset(result, d, 'Test accuracy')

    result = get_metric_by_dataset(d, 'metrics/eval_loss')
    plot_by_dataset(result, d, 'Test loss')


<ipython-input-8-9fb577a493bf>:12: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-8-9fb577a493bf>:12: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-8-9fb577a493bf>:12: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-8-9fb577a493bf>:12: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-8-9fb577a493bf>:12: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-8-9fb577a493bf>:12: UserWarning: Matplotlib is cur